# Importy

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import kagglehub
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight

# Przygotowanie danych

In [ ]:
# Pobranie datasetu i znalezienie właściwych ścieżek
print("Pobieranie datasetu HAM10000...")
path = kagglehub.dataset_download("kmader/skin-cancer-mnist-ham10000")
base_path = path[0]

# Znajdowanie pliku metadata
print("\nSzukanie pliku metadata...")
metadata_file = None
for root, dirs, files in os.walk(base_path):
    for file in files:
        if file == 'HAM10000_metadata.csv':
            metadata_file = os.path.join(root, file)
            print(f"Znaleziono plik metadata: {metadata_file}")
            break
    if metadata_file:
        break

# Wczytanie metadanych
print("\nWczytywanie metadanych...")
df = pd.read_csv(metadata_file)

# Znajdowanie katalogów z obrazami
print("\nSzukanie obrazów...")
images_dirs = []
for root, dirs, files in os.walk(base_path):
    for dir_name in dirs:
        if dir_name.startswith('HAM10000_images'):
            images_dirs.append(os.path.join(root, dir_name))

print(f"Znalezione katalogi z obrazami: {images_dirs}")

# Przygotowanie ścieżek do obrazów
def find_image_path(image_id):
    for dir_path in images_dirs:
        img_path = os.path.join(dir_path, f'{image_id}.jpg')
        if os.path.exists(img_path):
            return img_path
    return None

df['filepath'] = df['image_id'].apply(find_image_path)

# Usunięcie wierszy z brakującymi obrazami
df = df.dropna(subset=['filepath'])

# Wyświetl rozkład klas
print("\nRozkład klas w zbiorze:")
print(df['dx'].value_counts())

# Podział na zbiory treningowy, walidacyjny i testowy (60-20-20)
train_df, temp_df = train_test_split(df, test_size=0.4, stratify=df['dx'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['dx'], random_state=42)

print(f"\nRozmiary zbiorów:")
print(f"Rozmiar zbioru treningowego: {len(train_df)}")
print(f"Rozmiar zbioru walidacyjnego: {len(val_df)}")
print(f"Rozmiar zbioru testowego: {len(test_df)}")

# Parametry
img_size = (224, 224)
batch_size = 32

# Obliczenie wag klas (zachowujemy to dla lepszego uczenia)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_df['dx']),
    y=train_df['dx']
)
class_weight_dict = dict(zip(range(len(np.unique(train_df['dx']))), class_weights))

# Generator dla treningu z rozszerzoną augmentacją
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    brightness_range=[0.8,1.2],
    shear_range=0.2
)

# Generator dla walidacji i testu
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Generatory danych
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filepath',
    y_col='dx',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_dataframe(
    dataframe=val_df,
    x_col='filepath',
    y_col='dx',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = val_test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filepath',
    y_col='dx',
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Wyświetl rozkład klas w każdym zbiorze
print("\nRozkład klas w zbiorach:")
print("\nZbiór treningowy:")
print(train_df['dx'].value_counts())
print("\nZbiór walidacyjny:")
print(val_df['dx'].value_counts())
print("\nZbiór testowy:")
print(test_df['dx'].value_counts())

# Przykładowe zdjecia

In [ ]:
# Słownik z opisami klas
dx_descriptions = {
    'akiec': 'Rogowacenie słoneczne / Rak kolczystokomórkowy (in situ)',
    'bcc': 'Rak podstawnokomórkowy',
    'bkl': 'Łagodne zmiany keratotyczne (keratosis benigna)',
    'df': 'Włókniak twardy (dermatofibroma)',
    'nv': 'Znamię melanocytowe',
    'mel': 'Czerniak',
    'vasc': 'Zmiany naczyniowe'
}

# Pobierz po jednym przykładzie z każdej klasy
examples = {}
for dx in df['dx'].unique():
    example = df[df['dx'] == dx].iloc[0]
    examples[dx] = example['filepath']

# Utwórz wykres
plt.figure(figsize=(20, 10))

for idx, (dx, img_path) in enumerate(examples.items(), 1):
    try:
        img = Image.open(img_path)
        plt.subplot(2, 4, idx)
        plt.imshow(img)
        plt.title(f"{dx.upper()}\n{dx_descriptions[dx]}", fontsize=10)
        plt.axis('off')
    except Exception as e:
        print(f"Nie udało się załadować obrazu dla klasy {dx}: {e}")

plt.tight_layout()
plt.show()

# Wyświetl statystyki dla każdej klasy
print("\nLiczba przypadków w każdej klasie:")
class_counts = df['dx'].value_counts()
for dx in df['dx'].unique():
    print(f"\n{dx.upper()} - {dx_descriptions[dx]}")
    print(f"Liczba przypadków: {class_counts[dx]}")

# Model 1: Baseline CNN

In [ ]:
print("Trenowanie modelu CNN od podstaw...")

model_cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

model_cnn.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    mode='min',
    verbose=1
)

model_cnn.summary()

history_cnn = model_cnn.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    class_weight=class_weight_dict,
    callbacks=[early_stopping]
)

# Zapisz model
model_cnn.save('models/model_cnn.h5')

# Wizualizacja wyników dla modelu CNN

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_cnn.history['loss'], label='Strata treningowa')
plt.plot(history_cnn.history['val_loss'], label='Strata walidacyjna')
plt.title('Loss - CNN')
plt.xlabel('Epoka')
plt.ylabel('Strata')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_cnn.history['accuracy'], label='Dokładność treningowa')
plt.plot(history_cnn.history['val_accuracy'], label='Dokładność walidacyjna')
plt.title('Accuracy - CNN')
plt.xlabel('Epoka')
plt.ylabel('Dokładność')
plt.legend()

plt.tight_layout()
plt.show()

# Macierz pomyłek dla modelu CNN

In [ ]:
y_true = test_generator.classes
y_pred = model_cnn.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Macierz pomyłek
cm = confusion_matrix(y_true, y_pred_classes)
cmd = ConfusionMatrixDisplay(cm, display_labels=test_generator.class_indices.keys())
cmd.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix - CNN')
plt.show()

# Procentowa macierz pomyłek
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
cmd_percentage = ConfusionMatrixDisplay(cm_percentage, display_labels=test_generator.class_indices.keys())
cmd_percentage.plot(cmap=plt.cm.Blues)
plt.title('Percentage Confusion Matrix - CNN')
plt.show()

# Macierz pomyłek dla wczytanego modelu CNN

In [ ]:
# Wczytaj zapisany model
model_cnn = load_model('models/model_cnn.h5')

# Przewidywania dla zbioru testowego
y_true = test_generator.classes
y_pred = model_cnn.predict(test_generator)
y_pred_classes = np.argmax(y_pred, axis=1)

# Macierz pomyłek
cm = confusion_matrix(y_true, y_pred_classes)
cmd = ConfusionMatrixDisplay(cm, display_labels=test_generator.class_indices.keys())
cmd.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix - CNN')
plt.show()

# Procentowa macierz pomyłek
cm_percentage = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
cmd_percentage = ConfusionMatrixDisplay(cm_percentage, display_labels=test_generator.class_indices.keys())
cmd_percentage.plot(cmap=plt.cm.Blues)
plt.title('Percentage Confusion Matrix - CNN')
plt.show()

# Model 2: Transfer Learning z MobileNetV2

In [ ]:
print("Trenowanie modelu z Transfer Learningiem (MobileNetV2)...")

# Wczytanie pretrenowanego modelu MobileNetV2 (bez głowy klasyfikacyjnej)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))
base_model.trainable = False  # Zamrażamy warstwy MobileNetV2

model_tl = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Redukcja wymiarów wyjścia z MobileNetV2
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')
])

model_tl.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model_tl.summary()

history_tl = model_tl.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    epochs=25
)

# Zapisz model
model_tl.save('models/model_tl.h5')

# Wizualizacja wyników dla modelu z transfer learningiem

In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history_tl.history['loss'], label='Strata treningowa')
plt.plot(history_tl.history['val_loss'], label='Strata walidacyjna')
plt.title('Loss - Transfer Learning')
plt.xlabel('Epoka')
plt.ylabel('Strata')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_tl.history['accuracy'], label='Dokładność treningowa')
plt.plot(history_tl.history['val_accuracy'], label='Dokładność walidacyjna')
plt.title('Accuracy - Transfer Learning')
plt.xlabel('Epoka')
plt.ylabel('Dokładność')
plt.legend()

plt.tight_layout()
plt.show()

# Macierz pomyłek dla modelu z transfer learningiem

In [ ]:
# Przewidywania dla zbioru testowego
y_true_tl = test_generator.classes
y_pred_tl = model_tl.predict(test_generator)
y_pred_classes_tl = np.argmax(y_pred_tl, axis=1)

# Macierz pomyłek
cm_tl = confusion_matrix(y_true_tl, y_pred_classes_tl)
cmd_tl = ConfusionMatrixDisplay(cm_tl, display_labels=test_generator.class_indices.keys())
cmd_tl.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix - Transfer Learning')
plt.show()

# Procentowa macierz pomyłek
cm_percentage_tl = cm_tl.astype('float') / cm_tl.sum(axis=1)[:, np.newaxis] * 100
cmd_percentage_tl = ConfusionMatrixDisplay(cm_percentage_tl, display_labels=test_generator.class_indices.keys())
cmd_percentage_tl.plot(cmap=plt.cm.Blues)
plt.title('Percentage Confusion Matrix - Transfer Learning')
plt.show()

# Macierz pomyłek dla wczytanego modelu z transfer learningiem

In [ ]:
# Wczytaj zapisany model
model_tl = load_model('models/model_tl.h5')

# Przewidywania dla zbioru testowego
y_true_tl = test_generator.classes
y_pred_tl = model_tl.predict(test_generator)
y_pred_classes_tl = np.argmax(y_pred_tl, axis=1)

# Macierz pomyłek
cm_tl = confusion_matrix(y_true_tl, y_pred_classes_tl)
cmd_tl = ConfusionMatrixDisplay(cm_tl, display_labels=test_generator.class_indices.keys())
cmd_tl.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix - Transfer Learning')
plt.show()

# Procentowa macierz pomyłek
cm_percentage_tl = cm_tl.astype('float') / cm_tl.sum(axis=1)[:, np.newaxis] * 100
cmd_percentage_tl = ConfusionMatrixDisplay(cm_percentage_tl, display_labels=test_generator.class_indices.keys())
cmd_percentage_tl.plot(cmap=plt.cm.Blues)
plt.title('Percentage Confusion Matrix - Transfer Learning')
plt.show()